In [4]:
# import package needed
%matplotlib inline
import matplotlib.pyplot as plt
import os
os.environ["SDL_VIDEODRIVER"] = "dummy"  # this line make pop-out window not appear
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf
import numpy as np
import skimage.color
import skimage.transform
from ple.games.flappybird import FlappyBird
from ple import PLE
game = FlappyBird()
env = PLE(game, fps=30, display_screen=False)  # environment interface to game

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html
couldn't import doomish
Couldn't import doom


In [5]:
# return a dictionary whose key is action description and value is action index
print(game.actions)
# return a list of action index (include None)
num_action = len(env.getActionSet())
bucket_range_per_feature = {
  'next_next_pipe_bottom_y': 40,
  'next_next_pipe_dist_to_player': 512,
  'next_next_pipe_top_y': 40,
  'next_pipe_bottom_y': 20,
  'next_pipe_dist_to_player': 20,
  'next_pipe_top_y': 20,
  'player_vel': 4,
  'player_y': 16
}
print(num_action)
game.getGameState()  # a dictionary describe state

{'up': 119}
2


{'player_y': 256,
 'player_vel': 0,
 'next_pipe_dist_to_player': 309.0,
 'next_pipe_top_y': 29,
 'next_pipe_bottom_y': 129,
 'next_next_pipe_dist_to_player': 453.0,
 'next_next_pipe_top_y': 107,
 'next_next_pipe_bottom_y': 207}

In [6]:
# define input size
screen_width = 80
screen_height = 80
num_stack = 4
num_feature = 8

In [7]:
state_key = [k for k, v in sorted(bucket_range_per_feature.items())]
def preprocess(state):
    state = copy.deepcopy(state)
    state['next_next_pipe_bottom_y'] -= state['player_y']
    state['next_next_pipe_top_y'] -= state['player_y']
    state['next_pipe_bottom_y'] -= state['player_y']
    state['next_pipe_top_y'] -= state['player_y']
    state_idx = []
    for key in state_key:
        state_idx.append(int(state[key] / bucket_range_per_feature[key]))
    return state_idx

In [8]:
import math
import copy
from collections import defaultdict
MIN_EXPLORING_RATE = 10e-4


class Agent:
    def __init__(self, name, num_action, t=0, discount_factor=0.99):
        self.exploring_rate = 0.1
        self.discount_factor = discount_factor
        self.num_action = num_action
        self.name = name
        with tf.variable_scope(name):
            self.build_model()

    def build_model(self):

        # input: current screen, selected action and reward
        self.input_screen = tf.placeholder(
            tf.float32, shape=[None, num_feature, num_stack])
        self.action = tf.placeholder(tf.int32, [None])
        self.reward = tf.placeholder(tf.float32, [None])
        self.is_training = tf.placeholder(tf.bool, shape=[])

        def net(screen, reuse=False):
            with tf.variable_scope(
                    "layers",
                    reuse=reuse,
                    initializer=tf.truncated_normal_initializer(stddev=1e-2)):
                
                flat = tf.contrib.layers.flatten(screen)
                dense = tf.layers.dense(
                    inputs=flat, units=1024, activation=tf.nn.relu)
                dense = tf.layers.dense(
                    inputs=dense, units=512, activation=tf.nn.relu)
                dense = tf.layers.dense(
                    inputs=dense, units=256, activation=tf.nn.relu)
                dense = tf.layers.dense(
                    inputs=dense, units=128, activation=tf.nn.relu)
                Q = tf.layers.dense(
                    inputs=dense, units=self.num_action, activation=None)

                return Q

        # optimize
        self.output = net(
            self.input_screen
        )  # Q(s,a,theta) for all a, shape (batch_size, num_action)
        index = tf.stack([tf.range(tf.shape(self.action)[0]), self.action],
                         axis=1)
        self.esti_Q = tf.gather_nd(
            self.output,
            index)  # Q(s,a,theta) for selected action, shape (batch_size, 1)

        self.max_Q = tf.reduce_max(
            self.output, axis=1)  # max(Q(s',a',theta')), shape (batch_size, 1)
        self.tar_Q = tf.placeholder(tf.float32, [None])

        # loss = E[r+max(Q(s',a',theta'))-Q(s,a,theta)]
        self.loss = tf.reduce_mean(
            tf.square(self.reward + self.discount_factor * self.tar_Q -
                      self.esti_Q))

        optimizer = tf.train.AdamOptimizer(learning_rate=1e-5)
        self.g_gvs = optimizer.compute_gradients(
            self.loss,
            var_list=[v for v in tf.global_variables() if self.name in v.name])
        self.train_op = optimizer.apply_gradients(self.g_gvs)
        self.pred = tf.argmax(
            self.output, axis=1
        )  # select action with highest action-value, only used in inference

    def select_action(self, input_screen, sess):
        # epsilon-greedy
        if np.random.rand() < self.exploring_rate:
            action = np.random.choice(self.num_action)  # Select a random action
        else:
            input_screen = np.array(input_screen).transpose([1, 0])
            feed_dict = {
                self.input_screen: input_screen[None, :],
                self.is_training: False,
            }
            action = sess.run(
                self.pred,
                feed_dict=feed_dict)[0]  # Select the action with the highest q
        return action

    def update_policy(self, input_screens, actions, rewards,
                      input_screens_plum, terminal, target_netwrok, sess):
        # use max_Q estimate from target one to update online one
        feed_dict = {
            target_netwrok.input_screen:
            np.array(input_screens_plum).transpose([0, 2, 1]),
            target_netwrok.is_training:
            True,
        }
        max_Q = sess.run(target_netwrok.max_Q, feed_dict=feed_dict)
        max_Q *= ~np.array(terminal)
        feed_dict = {
            self.input_screen: np.array(input_screens).transpose([0, 2, 1]),
            self.tar_Q: max_Q,
            self.action: actions,
            self.reward: rewards,
            self.is_training: True,
        }
        loss, _ = sess.run([self.loss, self.train_op], feed_dict=feed_dict)
        return loss

    def update_parameters(self, episode):
        if self.exploring_rate > MIN_EXPLORING_RATE:
            self.exploring_rate -= (0.1 - MIN_EXPLORING_RATE) / 30000
#             self.exploring_rate = min(0.1, 0.99**((episode) / 30))

    def shutdown_explore(self):
        # make action selection greedy
        self.exploring_rate = 0


In [9]:
def get_update_ops():
  # return operations assign weight to target network
  src_vars = [v for v in tf.global_variables() if 'online' in v.name]
  tar_vars = [v for v in tf.global_variables() if 'target' in v.name]
  update_ops = []
  for src_var, tar_var in zip(src_vars, tar_vars):
    update_ops.append(tar_var.assign(src_var))
  return update_ops


def update_target(update_ops, sess):
  sess.run(update_ops)

In [10]:
# init agent
tf.reset_default_graph()
num_action = len(env.getActionSet())

# agent for frequently updating
online_agent = Agent('online', num_action)

# agent for slow updating
target_agent = Agent('target', num_action)
update_ops = get_update_ops()

In [11]:
class Replay_buffer():

  def __init__(self, buffer_size=50000):
    self.experiences = []
    self.buffer_size = buffer_size

  def add(self, experience):
    if len(self.experiences) >= self.buffer_size:
      self.experiences.pop(0)
    self.experiences.append(experience)

  def sample(self, size):
    """
        sameple experience from buffer
        """
    if size > len(self.experiences):
      experiences_idx = np.random.choice(len(self.experiences), size=size)
    else:
      experiences_idx = np.random.choice(
          len(self.experiences), size=size, replace=False)
    # from all sampled experiences, extract a tuple of (s,a,r,s')
    screens = []
    actions = []
    rewards = []
    screens_plum = []
    terminal = []
    for i in range(size):
      screens.append(self.experiences[experiences_idx[i]][0])
      actions.append(self.experiences[experiences_idx[i]][1])
      rewards.append(self.experiences[experiences_idx[i]][2])
      screens_plum.append(self.experiences[experiences_idx[i]][3])
      terminal.append(self.experiences[experiences_idx[i]][4])
    return screens, actions, rewards, screens_plum, terminal

In [12]:
# init buffer
buffer = Replay_buffer()

In [13]:
def make_anim(images, fps=60, true_image=False):
  duration = len(images) / fps
  import moviepy.editor as mpy

  def make_frame(t):
    try:
      x = images[int(len(images) / duration * t)]
    except:
      x = images[-1]

    if true_image:
      return x.astype(np.uint8)
    else:
      return ((x + 1) / 2 * 255).astype(np.uint8)

  clip = mpy.VideoClip(make_frame, duration=duration)
  clip.fps = fps
  return clip

In [14]:
# init all
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)
sess.run(tf.global_variables_initializer())

In [16]:
from IPython.display import Image, display

update_every_t_step = 3
print_every_episode = 10
save_video_every_episode = 100
NUM_EPISODE = 30000
NUM_EXPLORE = 20

# we can redefine origin reward function
reward_values = {
    "positive": 1,  # reward pass a pipe
    "tick": 0.1,  # reward per timestamp
    "loss": -1,  # reward of gameover
}

# Reset the environment
game = FlappyBird()
# for demo purpose, the following code is trained in the same scene,
env = PLE(
    game,
    fps=30,
    display_screen=False,
    reward_values=reward_values,
    rng=np.random.RandomState(1))

for episode in range(0, NUM_EPISODE + 1):

    env.reset_game()
    env.act(0)  # dummy input to make sure input screen is correct

    # record frame
    if episode % save_video_every_episode == 0:
        frames = [env.getScreenRGB()]

    # for every 500 episodes, shutdown exploration to see performance of greedy action
#     if episode % print_every_episode == 0:
#         online_agent.shutdown_explore()
#

    # grayscale input screen for this episode
    input_screens = [preprocess(game.getGameState())] * 4

    # experience for this episode, store all (s,a,r,s') tuple
    experience = []

    # cumulate reward for this episode
    cum_reward = 0

    t = 0
    while not env.game_over():

        # feed four previous screen, select an action
        action = online_agent.select_action(input_screens[-4:], sess)
        
        # execute the action and get reward
        reward = env.act(env.getActionSet()[action])

        # record frame
        if episode % save_video_every_episode == 0:
            frames.append(env.getScreenRGB())

        # cumulate reward
        cum_reward += reward

        # append grayscale screen for this episode
        input_screens.append(preprocess(game.getGameState()))

        # append experience for this episode
        buffer.add((input_screens[-5:-1], action, reward, input_screens[-4:],
                    env.game_over()))
        t += 1

        # update agent
    if episode > NUM_EXPLORE:
        train_screens, train_actions, train_rewards, train_screens_plum, terminal = buffer.sample(32)
        loss = online_agent.update_policy(train_screens, train_actions,
                                          train_rewards, train_screens_plum,
                                          terminal, target_agent, sess)
    if t % update_every_t_step == 0 and episode > NUM_EXPLORE:
        update_target(update_ops, sess)

    # update explore rating and learning rate
    online_agent.update_parameters(episode)
    target_agent.update_parameters(episode)

    if episode % print_every_episode == 0 and episode > NUM_EXPLORE:
        print(
            "[{}] time live:{}, cumulated reward: {}, exploring rate: {}, loss: {}"
            .format(episode, t, cum_reward, target_agent.exploring_rate, loss))

    if episode % save_video_every_episode == 0:  # for every 100 episode, record an animation
        clip = make_anim(frames, fps=60, true_image=True).rotate(-90)
        clip.write_videofile("mrr8iirovie/DQN-{}.webm".format(episode), fps=60)

[MoviePy] >>>> Building video movie/DQN-0.webm
[MoviePy] Writing video movie/DQN-0.webm


100%|██████████| 63/63 [00:00<00:00, 63.16it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: movie/DQN-0.webm 



[30] time live:61, cumulated reward: 5.099999999999994, exploring rate: 0.09989440000000008, loss: 0.03494509682059288
[40] time live:61, cumulated reward: 5.099999999999994, exploring rate: 0.0998614000000001, loss: 0.009967807680368423
[50] time live:60, cumulated reward: 4.999999999999995, exploring rate: 0.09982840000000012, loss: 0.00999574363231659
[60] time live:61, cumulated reward: 5.099999999999994, exploring rate: 0.09979540000000015, loss: 0.009932776913046837
[70] time live:61, cumulated reward: 5.099999999999994, exploring rate: 0.09976240000000017, loss: 0.009953608736395836
[80] time live:58, cumulated reward: 4.799999999999995, exploring rate: 0.09972940000000019, loss: 0.009977791458368301
[90] time live:55, cumulated reward: 4.4999999999999964, exploring rate: 0.09969640000000021, loss: 0.009904456324875355
[100] time live:61, cumulated reward: 5.099999999999994, exploring rate: 0.09966340000000024, loss: 0.03517235815525055
[MoviePy] >>>> Building video movie/DQN-10

100%|██████████| 63/63 [00:00<00:00, 64.32it/s]

...

 99%|█████████▉| 104/105 [00:01<00:00, 84.07it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: movie/DQN-29900.webm 



[29910] time live:97, cumulated reward: 9.699999999999982, exploring rate: 0.0012904000000419978, loss: 0.1923304945230484
[29920] time live:61, cumulated reward: 5.099999999999994, exploring rate: 0.0012574000000419986, loss: 0.12453856319189072
[29930] time live:61, cumulated reward: 5.099999999999994, exploring rate: 0.0012244000000419994, loss: 0.09767059981822968
[29940] time live:64, cumulated reward: 5.399999999999993, exploring rate: 0.0011914000000420003, loss: 0.17575593292713165
[29950] time live:97, cumulated reward: 9.699999999999982, exploring rate: 0.001158400000042001, loss: 0.05575591325759888
[29960] time live:97, cumulated reward: 9.699999999999982, exploring rate: 0.001125400000042002, loss: 0.03229641169309616
[29970] time live:61, cumulated reward: 5.099999999999994, exploring rate: 0.0010924000000420027, loss: 0.039504025131464005
[29980] time live:217, cumulated reward: 25.70000000000008, exploring rate: 0.0010594000000420036, loss: 0.033990100026130676
[29990] 

 99%|█████████▊| 76/77 [00:00<00:00, 78.69it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: movie/DQN-30000.webm 



In [17]:
from moviepy.editor import *
clip = VideoFileClip("movie/DQN-29700.webm")
display(clip.ipython_display(fps=60, autoplay=1, loop=1))

100%|██████████| 935/935 [00:02<00:00, 466.90it/s]


### DiScusS ThE rEsuLt

Training DQN is faster than the previous lab(e.g. sarsa) and the result after training is also better.
Using state to represent the game is better than using screen since it cost lower memory to store it.
